# Mock code for preparing and loading data for training espaloma

## Download QC datasets from QCArchive as HDF5 (SKIP IMPLEMENTATION) 

This functionality will not be implemented at the moment and alternatively rely on external scripts (e.g. https://github.com/choderalab/download-qca-datasets).

## Convert HDF5 to DGL graphs (SKIP IMPLEMENTATION)

This function will not be implemented at the moment and alternatively rely on external scripts (e.g. https://github.com/choderalab/refit-espaloma/blob/main/openff-default/01-create-dataset/script/getgraph_hdf5.py).

## Filter DGL graphs (SKIP IMPLEMENTATION)

This function will not be implemented at the moment and rely on external scripts (e.g. https://github.com/choderalab/refit-espaloma/tree/main/openff-default/02-train/merge-data/script).

## Load preprocessed DGL graphs

#### Check properties

#### Drop/merge duplicate smiles and filter datasets

Ensure the datasets loaded from different sources have no duplicated smiles.  
Drop duplicate isomeric (nonisomeric) smiles across different sources of datasets.  
Merge duplicate dgl graphs with same smiles into a single dgl graph and create a new dataset called 'misc'.

##### drop and merge smiles

##### filter dataset

## Prepare for training

#### Split datasets

#### Augment conformations to handle heterographs

This is a work around to handle different graph size (shape). DGL requires at least one dimension with same size. 
Here, we will modify the graphs so that each graph has the same number of conformations instead fo concatenating 
graphs into heterogenous graphs with the same number of conformations. This will allow batching and shuffling 
during the training. 

## Train espaloma

#### Validate and find best model

Use job array to speed up this process using external scripts (e.g. https://github.com/choderalab/refit-espaloma/tree/main/openff-default/02-train/joint-improper-charge/charge-weight-1.0/eval)

## Alternatively, train and validate simultaneously

Not sure how slower this will be compared to just doing trainig

## Benchmark

#### RMSE metric

#### Run other benchmarks independantly.

- Small molecule geometry optmization (https://github.com/choderalab/geometry-benchmark-espaloma/tree/main/qc-opt-geo)
- ESP benchmark

## Train espaloma with experimental observable refitting

- `espfit_experiment/`
    - `data/`: Cached dataset ready for training
    - `utils/`: Stores scripts to run external benchmarks
        - `small_molecule_geometry`
            - geo.py
        - `partial_charge_esp`
            - ele.py
        - `rna_nucleoside`
            - rna_nucleoside.py
        - `rna_tetramer`:
            - rna_tetramer.py
    - `experiment/`
        - `001/`: Create new directory for each refitting experiment
            - `xml/`: Stores openmm xml
            - `refit/`: Espaloma training
                - `checkpoints/`: Stores checkpoint files
                - `sampling/`: MD simulation
                    - `iter_0`: Initial MD sampling
                    - `iter_n`: MD sampling at epoch-n when necesssary
                - `train.log`: Log file during espaloma training
            - `benchmark/`
                - `rmse_metric`
                - `small_molecule_geometry`
                - `partial_charge_esp`
                - `rna_nucleoside`
                - `rna_tetramer`

#### Basic usage to run simulations for registered systems

##### Prepare system

##### Load a system already prepared

##### Run simulation

##### Compute loss

##### Reweight observable using updated espaloma model

## Pseudo code for training espaloma with reweighting on the fly

##### Run simulation

##### Get experimental observables

##### Train with MD reweighting

#### Wrapper